In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt

from metpy.calc import dewpoint_from_relative_humidity
from metpy.units import units
from metpy.plots import SkewT

from sounding_utils import *
from importlib import reload

from functools import partial

import sys
sys.path.append('../../') # lets us import ptype package from the subdir

#import ptype.

from dask.distributed import Client, LocalCluster
from dask_jobqueue import PBSCluster
import dask
import glob
from os.path import join

In [64]:
cluster = PBSCluster(account='NAML0001',
                     queue='cheyenne',
                     walltime='01:00:00',
                     memory="200 GB",
                     cores=24,
                     interface='ib0')

# Change your url to the dask dashboard so you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})
                    
client = Client(cluster)
cluster.scale(jobs=10, memory="200GB")
#client.shutdown()
cluster

/glade/work/dkimpara/conda-envs/ptype/lib/python3.10/site-packages/dask_jobqueue/pbs.py:82: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config files). If you did not set account yet, project will be respected for now, but it will be removed in a future release. If you already set account, project is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/glade/work/dkimpara/conda-envs/ptype/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37015 instead
  warnings.warn(


Use this link to monitor the workload: https://jupyterhub.hpc.ucar.edu/stable/user/dkimpara/pux/proxy/37015/status


In [65]:
a = glob.glob("/glade/campaign/cisl/aiml/ptype/ptype_case_studies/kentucky/rap/20220223/*/*.nc")

files = a

In [ ]:
%%time
ds = xr.open_mfdataset(files, parallel=True, engine='netcdf4', 
                       decode_cf=False, concat_dim='time', combine='nested', 
                       chunks={'time':1, 'heightAboveGround': 21, 'y':337, 'x': 451, 'isobaricInhPa': 37})